# Simulation

The `Simulation` class is the centerpiece of most TRIPOLI-5 calculations. It collects and coordinates all the necessary parameters and information, which makes it a rather complex object. For this reason, TRIPOLI-5 provides a `SimulationBuilder` class that helps the user construct their `Simulation` object.

The purpose of this notebook is to illustrate the different options available in `SimulationBuilder`. For this purpose, we will concentrate on a simple use case: a uniform box. We start with a few imports:

In [1]:
%load_ext wurlitzer

In [2]:
from tripoli5.core.euclide import Direction, Point
from tripoli5.core.literals import K, MeV
import tripoli5

## The system

We set up a few important objects, such as the [geometry](agora_geometry.ipynb), the [materials](materials_compositions.ipynb), the [scores](score.ipynb), and the [source](source.ipynb). We will not discuss this part in detail; take a look at the other notebooks if want to learn more.

### Mixtures

In [3]:
catalog = tripoli5.delos.Catalog.fromFile(
    f"/opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/endfb8r0_mcnp_v{tripoli5.delos.CATALOG_VERSION}.yaml"
)

uranium_mix = (
    tripoli5.materials.MixtureBuilder("concentrations")
    .withName("uranium")
    .withCatalog(catalog)
    .withTemperature(294 * K)
    .add("U234", 4.8416e-05)
    .add("U235", 4.8151e-03)
    .add("U236", 1.7407e-05)
    .add("U238", 4.3181e-02)
    .build()
)

[2025-09-03 12:43:43.939] [info] Loading catalog from file: /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/endfb8r0_mcnp_v2.2.0.yaml
[2025-09-03 12:43:43.970] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/mass/njoy_ace/v0.2.0/archive/wrapped_mass_data.bin


### Geometry

In [4]:
# define the dimensions
side = 15

# planes
plane_xmin = tripoli5.agora.XPlane(-0.5 * side, label="plane_xmin")
plane_xmax = tripoli5.agora.XPlane(0.5 * side, label="plane_xmax")
plane_ymin = tripoli5.agora.YPlane(-0.5 * side, label="plane_ymin")
plane_ymax = tripoli5.agora.YPlane(0.5 * side, label="plane_ymax")
plane_zmin = tripoli5.agora.ZPlane(-0.5 * side, label="plane_zmin")
plane_zmax = tripoli5.agora.ZPlane(0.5 * side, label="plane_zmax")

# shapes
box_region = (
    plane_xmin & -plane_xmax & plane_ymin & -plane_ymax & plane_zmin & -plane_zmax
)
box_shape = tripoli5.agora.Shape(box_region)
outside_shape = tripoli5.agora.Shape(-box_region)

# volumes
box_volume = tripoli5.agora.Volume(box_shape, "uranium", label="box")
outside = tripoli5.agora.Volume.outside(outside_shape, label="outside")

root_universe = tripoli5.agora.Universe([box_volume, outside])
geometry = tripoli5.agora.Geometry(root_universe)

### Media

In [5]:
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withCatalog(catalog)
    .withMixtures([uranium_mix])
    .build()
)

[2025-09-03 12:43:44.012] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:43:44.015] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-03 12:43:44.032] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:43:44.033] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-03 12:43:44.033] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:43:44.034] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-03 12:43:44.099] [info]   Building free gas colli

### Scores

In [6]:
flux = tripoli5.score.FluxSpectrumCollGeneral(
    [1e-11, 0.625e-6, 20.0], "FluxSpectrumCollGeneral"
)

score_mgr = tripoli5.score.ScoreManager("box")
score_mgr.addScore(flux)
score_mgr.initialize("")

### Source

In [7]:
src = tripoli5.source.FactorizedSource(
    tripoli5.distributions.spatial.PointDistribution(Point(1.0, 1.0, 1.0)),
    tripoli5.distributions.energy.MonoKinetic(2.0 * MeV),
    tripoli5.distributions.angular.IsotropicAngularDistribution(),
    1.0,  # norm
)

## Simulation parameters

The most important simulation parameters, such as the number of particles and the number of cycles, are stored in a `SimulationParameters` object. It is relatively straightforward:

In [8]:
param = tripoli5.simulation.SimulationParameters(
    nb_cycles=10,
    nb_events=1000,
    name="box",
)

Here:

- `nb_cycles` is the total number of cycles (batches).
- `nb_events` is the number of events (source particles) per cycle.
- `name` is just a string that helps you identify the simulation.

### Advanced Simulation parameters

More advanced parameters can be controled using the simulation parameters.

- `bundle_size` : this integer controls the number of particle histories simulated between tallying events. Indeed, during the simulation, particle tracks are recorded and accumulated; after a certain number of histories, the scores consume the tracks and free up their memory. Smaller values of this parameter help with mitigating the memory occupation due to this strategy.

- `random_seed` : change the random seed of the simulation.


## The simulation builder

With all the setup out of the way, we can now turn to the `SimulationBuilder` class. Its idea is similar to other builder classes used by TRIPOLI-5: you start by creating a builder, you add options to it, and you call `.build()` at the end to retrieve a `Simulation` object. For example, here is a very simple fixed-source simulation, in mono-processor mode:

In [9]:
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="mono"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .build()
)
simu.run()

[2025-09-03 12:43:44.255] [info] *** Beginning of simulation ***
[2025-09-03 12:43:44.255] [info] Cycle: 1 of 10
[2025-09-03 12:43:44.329] [info]   Mean number of collisions: 9.8770
[2025-09-03 12:43:44.329] [info] Cycle: 2 of 10
[2025-09-03 12:43:44.388] [info]   Mean number of collisions: 9.8320
[2025-09-03 12:43:44.388] [info] Cycle: 3 of 10
[2025-09-03 12:43:44.448] [info]   Mean number of collisions: 10.0720
[2025-09-03 12:43:44.448] [info] Cycle: 4 of 10
[2025-09-03 12:43:44.514] [info]   Mean number of collisions: 9.7640
[2025-09-03 12:43:44.514] [info] Cycle: 5 of 10
[2025-09-03 12:43:44.583] [info]   Mean number of collisions: 10.6770
[2025-09-03 12:43:44.583] [info] Cycle: 6 of 10
[2025-09-03 12:43:44.650] [info]   Mean number of collisions: 9.7670
[2025-09-03 12:43:44.650] [info] Cycle: 7 of 10
[2025-09-03 12:43:44.711] [info]   Mean number of collisions: 10.1620
[2025-09-03 12:43:44.711] [info] Cycle: 8 of 10
[2025-09-03 12:43:44.775] [info]   Mean number of collisions: 10.

We can run the same simulation in parallel, in shared-memory mode, by changing the `parallelism_type` argument to `openmp`. We will also need to specify the number of threads to use via the `SimulationParameters` object:

In [10]:
param = tripoli5.simulation.SimulationParameters(
    nb_cycles=10,
    nb_events=1000,
    name="box",
    nb_threads=20,
)
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .build()
)
simu.run()

[2025-09-03 12:43:44.950] [info] *** Beginning of simulation ***
[2025-09-03 12:43:44.950] [info] Cycle: 1 of 10
[2025-09-03 12:43:45.039] [info]   Mean number of collisions: 9.8770
[2025-09-03 12:43:45.040] [info] Cycle: 2 of 10
[2025-09-03 12:43:45.050] [info]   Mean number of collisions: 9.8320
[2025-09-03 12:43:45.050] [info] Cycle: 3 of 10
[2025-09-03 12:43:45.060] [info]   Mean number of collisions: 10.0720
[2025-09-03 12:43:45.060] [info] Cycle: 4 of 10
[2025-09-03 12:43:45.070] [info]   Mean number of collisions: 9.7640
[2025-09-03 12:43:45.070] [info] Cycle: 5 of 10
[2025-09-03 12:43:45.080] [info]   Mean number of collisions: 10.6770
[2025-09-03 12:43:45.081] [info] Cycle: 6 of 10
[2025-09-03 12:43:45.091] [info]   Mean number of collisions: 9.7670
[2025-09-03 12:43:45.091] [info] Cycle: 7 of 10
[2025-09-03 12:43:45.103] [info]   Mean number of collisions: 10.1620
[2025-09-03 12:43:45.103] [info] Cycle: 8 of 10
[2025-09-03 12:43:45.113] [info]   Mean number of collisions: 10.

Much faster! Now let's change the type of simulation to `criticality`. In criticality calculations, we probably also want to specify a certain number of inactive cycles, to allow the fission source to relax to its asymptotic distribution. In this example, we just use 10 inactive cycles and 20 active cycles:

In [11]:
param = tripoli5.simulation.SimulationParameters(
    nb_cycles=30,
    discard=10,
    nb_events=1000,
    name="box",
    nb_threads=20,
)
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="criticality", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .build()
)
simu.run()

[2025-09-03 12:43:45.152] [info] *** Beginning of simulation ***
[2025-09-03 12:43:45.153] [info] Cycle: 1 of 30
[2025-09-03 12:43:45.158] [info]   kstep = 0.51000
[2025-09-03 12:43:45.158] [info]   Mean number of collisions: 6.0750
[2025-09-03 12:43:45.158] [info] Cycle: 2 of 30
[2025-09-03 12:43:45.164] [info]   kstep = 0.36500
[2025-09-03 12:43:45.164] [info]   Mean number of collisions: 5.3480
[2025-09-03 12:43:45.164] [info] Cycle: 3 of 30
[2025-09-03 12:43:45.169] [info]   kstep = 0.32700
[2025-09-03 12:43:45.169] [info]   Mean number of collisions: 4.6500
[2025-09-03 12:43:45.169] [info] Cycle: 4 of 30
[2025-09-03 12:43:45.174] [info]   kstep = 0.29800
[2025-09-03 12:43:45.174] [info]   Mean number of collisions: 4.8000
[2025-09-03 12:43:45.174] [info] Cycle: 5 of 30
[2025-09-03 12:43:45.178] [info]   kstep = 0.32500
[2025-09-03 12:43:45.178] [info]   Mean number of collisions: 4.4770
[2025-09-03 12:43:45.178] [info] Cycle: 6 of 30
[2025-09-03 12:43:45.183] [info]   kstep = 0.30

Note that criticality simulation objects have a `KStep` score:

In [12]:
import math

kstep = simu.getKStep()
print(f"k_eff = {kstep.mean[0]:.5f} ± {math.sqrt(kstep.var[0]):.5f}")

k_eff = 0.33080 ± 0.00726


## Particle tracking

An important feature of the simulation builder is that it can be used to fine-tune the particle transport algorithm. Since particle tracking is also a complex beast, it has its own builder class! It is called `TrackingBuilder`. 

First, let's go back to the parameters for a small fixed-source simulation:

In [13]:
param = tripoli5.simulation.SimulationParameters(
    nb_cycles=10,
    nb_events=1000,
    name="box",
    nb_threads=20,
)

### Surface tracking and delta tracking

By default, _surface tracking_ is used. In some situations, however, you may want to use _delta tracking_ instead. Delta tracking is useful, in particular, when the cross section has a continuous dependence on the position, because of a continuous temperature or density field.

Let's construct a particle tracking configuration that uses surface tracking:

In [14]:
surface_tracking = tripoli5.simulation.TrackingBuilder().build()

That was easy! Surface tracking does not have a lot of free parameters. We can pass the tracking configuration to the `SimulationBuilder` with the `withParticleTracking()` method. (Note that, in the following, we are not systematically running all the simulations.)

In [15]:
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", surface_tracking)
    .build()
)

Delta tracking requires a bit more configuration. First, you have to decide what kind of majorant cross section you want to use. There are two options:

- `Global`: this corresponds to using a global majorant cross section value.
- `EnergyDependent`: unsurprisingly, this corresponds to an energy-dependent majorant cross section.

The `TrackingBuilder` tries to help with the calculation of the majorant cross section. Here is how it works:

In [16]:
global_delta_tracking = (
    tripoli5.simulation.TrackingBuilder(tripoli5.simulation.MajorantType.Global)
    .withMajorantParameters(media, tripoli5.simulation.MajorantExecutor.OpenMP)
    .build()
)

[2025-09-03 12:43:45.392] [info] Building majorant cross section...
[2025-09-03 12:43:45.400] [info] ... sorting the unionized grid...
[2025-09-03 12:43:45.416] [info] ... removing duplicate points from the grid...
[2025-09-03 12:43:45.417] [info] ... removing energies outside the simulation range...
[2025-09-03 12:43:45.420] [info] ... unionized grid has 270614 points...
[2025-09-03 12:43:45.642] [info] ... majorant cross section built!


In this example, a *global* majorant is computed, based on the media in our simulation. In case the media contain temperature or density fields, `TrackingBuilder` tries to maximize the total macroscopic cross section by taking the worst-case scenario of maximum density and minimum temperature everywhere. This works 95% of the time.

Since this calculation can be expensive, you can let `TrackingBuilder` know that you would like to run the calculation of the majorant in parallel, using `tripoli5.simulation.MajorantExecutor.OpenMP`.

Let's try to use delta tracking in a simulation: 

In [17]:
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", global_delta_tracking)
    .build()
)
simu.run()

[2025-09-03 12:43:45.663] [info] *** Beginning of simulation ***
[2025-09-03 12:43:45.663] [info] Cycle: 1 of 10
[2025-09-03 12:43:48.853] [info]   Mean number of collisions: 9.9320
[2025-09-03 12:43:48.854] [info] Cycle: 2 of 10
[2025-09-03 12:43:52.142] [info]   Mean number of collisions: 10.5480
[2025-09-03 12:43:52.142] [info] Cycle: 3 of 10
[2025-09-03 12:43:55.382] [info]   Mean number of collisions: 10.6170
[2025-09-03 12:43:55.382] [info] Cycle: 4 of 10
[2025-09-03 12:43:58.664] [info]   Mean number of collisions: 9.8590
[2025-09-03 12:43:58.664] [info] Cycle: 5 of 10
[2025-09-03 12:44:01.792] [info]   Mean number of collisions: 10.6630
[2025-09-03 12:44:01.792] [info] Cycle: 6 of 10
[2025-09-03 12:44:05.122] [info]   Mean number of collisions: 10.3010
[2025-09-03 12:44:05.122] [info] Cycle: 7 of 10
[2025-09-03 12:44:08.557] [info]   Mean number of collisions: 10.0710
[2025-09-03 12:44:08.557] [info] Cycle: 8 of 10
[2025-09-03 12:44:11.929] [info]   Mean number of collisions: 1

If you're following along and running the cells as you go, you may have noticed that the calculation with delta tracking is much slower than surface tracking! That is usually the case, because particles make shorter flights on average in delta tracking.

Let's try an energy-dependent majorant now:

In [18]:
edep_delta_tracking = (
    tripoli5.simulation.TrackingBuilder(
        tripoli5.simulation.MajorantType.EnergyDependent
    )
    .withMajorantParameters(media, tripoli5.simulation.MajorantExecutor.OpenMP)
    .withMajorantFudgeFactors([1e-11, 20.0], [1.1])
    .build()
)

[2025-09-03 12:44:18.446] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:18.446] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:44:18.447] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-03 12:44:18.447] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:18.447] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:44:18.447] [info]   Building reaction MTs = 814, 52, 813, 51, 818, 56, 76, 17, 817, 55, 75, 16, 77, 18, 816, 54, 72, 834, 815, 53, 822, 60, 821, 59, 5, 7

In [19]:
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", edep_delta_tracking)
    .build()
)
simu.run()

[2025-09-03 12:44:18.583] [info] *** Beginning of simulation ***
[2025-09-03 12:44:18.583] [info] Cycle: 1 of 10
[2025-09-03 12:44:18.600] [info]   Mean number of collisions: 9.7140
[2025-09-03 12:44:18.600] [info] Cycle: 2 of 10
[2025-09-03 12:44:18.613] [info]   Mean number of collisions: 10.7050
[2025-09-03 12:44:18.613] [info] Cycle: 3 of 10
[2025-09-03 12:44:18.625] [info]   Mean number of collisions: 10.4370
[2025-09-03 12:44:18.625] [info] Cycle: 4 of 10
[2025-09-03 12:44:18.635] [info]   Mean number of collisions: 9.3680
[2025-09-03 12:44:18.635] [info] Cycle: 5 of 10
[2025-09-03 12:44:18.647] [info]   Mean number of collisions: 10.2580
[2025-09-03 12:44:18.647] [info] Cycle: 6 of 10
[2025-09-03 12:44:18.658] [info]   Mean number of collisions: 10.3490
[2025-09-03 12:44:18.658] [info] Cycle: 7 of 10
[2025-09-03 12:44:18.669] [info]   Mean number of collisions: 10.5240
[2025-09-03 12:44:18.669] [info] Cycle: 8 of 10
[2025-09-03 12:44:18.680] [info]   Mean number of collisions: 1

This is fast again! We had to take some safety margins, though: the fudge factor of 1.1 means that `TrackingBuilder` will try to compute the optimal majorant cross section, but multiply it by 1.1 between $10^{-11}$ and $20$ MeV. A small fudge factor is usually necessary when working with energy-dependent majorants.

### Collision Sampling Algorithm

All the examples shown above use analog sampling (see the options of the defined media).  
Let's take a look at how to build a surface tracking object with _nuclide-wise implicit capture_. However, by applying implicit capture (or branchless), particles may become immortal, as the capture channels cannot be sampled. Thus, it forces us to apply population control. Here is how it may be defined:  

In [20]:
implicit_capture_media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withCatalog(catalog)
    .withMixtures([uranium_mix])
    .withCollisionSampler("implicit_capture")
    .build()
)
implicit_capture_tracking = (
    tripoli5.simulation.TrackingBuilder().withPopulationControl().build()
)

[2025-09-03 12:44:18.714] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:18.718] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-03 12:44:18.734] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:44:18.734] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-03 12:44:18.734] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:18.735] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-03 12:44:18.796] [info]   Building free gas colli

Then we can run the simulation

In [21]:
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", implicit_capture_media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", implicit_capture_tracking)
    .build()
)
simu.run()

[2025-09-03 12:44:18.902] [info] *** Beginning of simulation ***
[2025-09-03 12:44:18.902] [info] Cycle: 1 of 10
[2025-09-03 12:44:18.932] [info]   Mean number of collisions: 11.4140
[2025-09-03 12:44:18.932] [info] Cycle: 2 of 10
[2025-09-03 12:44:18.949] [info]   Mean number of collisions: 11.5390
[2025-09-03 12:44:18.949] [info] Cycle: 3 of 10
[2025-09-03 12:44:18.963] [info]   Mean number of collisions: 11.4870
[2025-09-03 12:44:18.963] [info] Cycle: 4 of 10
[2025-09-03 12:44:18.977] [info]   Mean number of collisions: 11.2730
[2025-09-03 12:44:18.977] [info] Cycle: 5 of 10
[2025-09-03 12:44:18.992] [info]   Mean number of collisions: 11.9020
[2025-09-03 12:44:18.992] [info] Cycle: 6 of 10
[2025-09-03 12:44:19.006] [info]   Mean number of collisions: 11.3380
[2025-09-03 12:44:19.006] [info] Cycle: 7 of 10
[2025-09-03 12:44:19.021] [info]   Mean number of collisions: 11.4190
[2025-09-03 12:44:19.021] [info] Cycle: 8 of 10
[2025-09-03 12:44:19.035] [info]   Mean number of collisions:

Selecting the branchless mode can be performed as follows:

In [22]:
branchless_media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withCatalog(catalog)
    .withMixtures([uranium_mix])
    .withCollisionSampler("branchless")
    .build()
)
branchless_tracking = (
    tripoli5.simulation.TrackingBuilder().withPopulationControl().build()
)
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", branchless_media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", branchless_tracking)
    .build()
)
simu.run()

[2025-09-03 12:44:19.076] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:19.081] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-03 12:44:19.095] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:44:19.095] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-03 12:44:19.096] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:19.098] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-03 12:44:19.172] [info]   Building free gas colli

### Population control parameters

Population control has the following parameters: 
- `roulette_threshold`: threshold below which Russian roulette is applied (default = 0.25)
- `roulette_survival_weight`: weight of particle surviving the Russian roulette (default = 1.0) 
- `splitting_threshold`: threshold above which splitting is applied (default = 4.0)
- `splitting_target_weight`: average weight of particles after splitting (default = 1.0)

However roulette and splitting parameters can be specified as below:

In [23]:
pop_control_tracking = (
    tripoli5.simulation.TrackingBuilder()
    .withPopulationControl(
        roulette_threshold=0.1,
        roulette_survival_weight=0.2,
        splitting_threshold=10,
        splitting_target_weight=1.0,
    )
    .build()
)
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="openmp"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", implicit_capture_media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", pop_control_tracking)
    .build()
)
simu.run()

[2025-09-03 12:44:19.451] [info] *** Beginning of simulation ***
[2025-09-03 12:44:19.451] [info] Cycle: 1 of 10
[2025-09-03 12:44:19.465] [info]   Mean number of collisions: 11.4330
[2025-09-03 12:44:19.465] [info] Cycle: 2 of 10
[2025-09-03 12:44:19.477] [info]   Mean number of collisions: 11.5730
[2025-09-03 12:44:19.477] [info] Cycle: 3 of 10
[2025-09-03 12:44:19.491] [info]   Mean number of collisions: 11.4870
[2025-09-03 12:44:19.491] [info] Cycle: 4 of 10
[2025-09-03 12:44:19.504] [info]   Mean number of collisions: 11.0880
[2025-09-03 12:44:19.504] [info] Cycle: 5 of 10
[2025-09-03 12:44:19.518] [info]   Mean number of collisions: 11.7590
[2025-09-03 12:44:19.518] [info] Cycle: 6 of 10
[2025-09-03 12:44:19.532] [info]   Mean number of collisions: 11.4960
[2025-09-03 12:44:19.532] [info] Cycle: 7 of 10
[2025-09-03 12:44:19.545] [info]   Mean number of collisions: 11.3940
[2025-09-03 12:44:19.545] [info] Cycle: 8 of 10
[2025-09-03 12:44:19.559] [info]   Mean number of collisions:

### Advanced parameters

There are a few more advanced tracking parameters that you may want to play around with, especially in very large calculations. One of them is `catchExceptions()`, which tells particle tracking to keep going, even in the presence of errors during transport. For example, let's take the previous energy-dependent simulation. Note the very small number of particles!

In [24]:
param = tripoli5.simulation.SimulationParameters(
    nb_cycles=1,
    nb_events=10,
    name="box",
)

edep_delta_tracking = (
    tripoli5.simulation.TrackingBuilder(
        tripoli5.simulation.MajorantType.EnergyDependent
    )
    .withMajorantParameters(media, tripoli5.simulation.MajorantExecutor.OpenMP)
    .catchExceptions(True)
    .build()
)

[2025-09-03 12:44:19.600] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:19.600] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:44:19.600] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-03 12:44:19.600] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-03 12:44:19.600] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-03 12:44:19.600] [info]   Building reaction MTs = 814, 52, 813, 51, 818, 56, 76, 17, 817, 55, 75, 16, 77, 18, 816, 54, 72, 834, 815, 53, 822, 60, 821, 59, 5, 7

In [25]:
simu = (
    tripoli5.simulation.SimulationBuilder(
        simulation_mode="fixed_source", parallelism_type="mono"
    )
    .withSource(src)
    .withGeometry(geometry)
    .withMedia("Neutron", media)
    .withParameters(param)
    .withScores(score_mgr)
    .withParticleTracking("neutron", edep_delta_tracking)
    .build()
)
simu.run()

[2025-09-03 12:44:19.708] [info] *** Beginning of simulation ***
[2025-09-03 12:44:19.708] [info] Cycle: 1 of 1
[2025-09-03 12:44:19.709] [error] Exception raised during transport. The particle will be killed. Exception: Could not evaluate collision at energy 0.8552740268826469 [MeV]
Method: particleCollisionStep, File: /builds/tripoli-5/tripoli-5/PATMOS/src/simulation/deltaparticletracking.cxx, Line: 159
[2025-09-03 12:44:19.709] [error] Exception raised during transport. The particle will be killed. Exception: Could not evaluate collision at energy 1.999273494252602 [MeV]
Method: particleCollisionStep, File: /builds/tripoli-5/tripoli-5/PATMOS/src/simulation/deltaparticletracking.cxx, Line: 159
[2025-09-03 12:44:19.709] [error] Exception raised during transport. The particle will be killed. Exception: Could not evaluate collision at energy 0.1084700230628964 [MeV]
Method: particleCollisionStep, File: /builds/tripoli-5/tripoli-5/PATMOS/src/simulation/deltaparticletracking.cxx, Line: 15

As you can see, some errors were generated, but the simulation kept running and just discarded the problematic particles. This can be useful for very long running simulations, which run the risk of hitting a rare bug. You don't want your massively parallel simulation to die because one particle out of $10^{12}$ ran into some weird edge case.